This Colab provides an example of setting up a simulated dataset. Please check [Meridian_RF_Demo.ipynb](https://github.com/google/meridian/blob/main/demo/Meridian_RF_Demo.ipynb) to see how to run a Google's Meridian MMM using the simulated dataset.

Throughout this Colab, we use the N(mean, stdev) parameterization for the normal distribution.

# Install and import packages

1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

In [ ]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda]

# Install meridian: from PyPI @ specific version
# !pip install google-meridian[colab,and-cuda]==1.0.3

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda] @ git+https://github.com/google/meridian.git"

In [ ]:
import datetime
import os

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import xarray as xr
import pickle

from pathlib import Path

from meridian import constants
from meridian.data import load
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.model import transformers
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter
from meridian.model import knots


# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

# Simulation setup

In [ ]:
# @title Specify simulation parameters {form-width:'400px'}

n_imp_channels = 3  # @param{type:'number'}
n_rf_channels = 1  # @param{type:'number'}

assert n_imp_channels > 0 and n_rf_channels > 0, 'Number of channels must be positive'

n_total_paid_channels = n_imp_channels + n_rf_channels

n_controls = 2  # @param{type:'number'}
n_times = 156  # @param{type:'number'}
n_geos = 20  # @param{type:'number'}

seed_num = 1320  # @param{type:'number'}
tf.random.set_seed(seed_num)

Explanation:
* n_imp_channels: number of impression-based (paid) media channels
* n_rf_channels: number of RF-based (paid) media channels
* n_controls: number of controls
* n_times: number of time periods
* n_geos: number of geos
* seed_num: seed number

## Population

Simulate population $p_g$ for each DMA $g$:

$$p_g \overset{\mathrm{iid}}{\sim} uniform(10^5,10^6)$$

In [ ]:
# @title Simulate population for each DMA

p_g = tfp.distributions.Uniform(1e5, 1e6).sample(n_geos)
# For broadcasting
p_gtm = tf.reshape(p_g, (-1, 1, 1))

The control variables $\ddot{z}_{g,t,c}$ are simulated first to account for its potential influence as a confounding factor on both media spending and sales. For each geo $g$, time period $t$ and control variable $c$,

$$\ddot{z}_{g,t,c} \stackrel{\text{iid}}{\sim} N(0,3) $$


In [ ]:
# @title Simulate control variables

control_gtc = tfp.distributions.Normal(0, 3).sample(
    [n_geos, n_times, n_controls]
)

To ensure that the control variables and media imressions are simulated at the same scale, we will standardize the control variables. We use the notation $z_{g,t,c}$ to represent the transformed control variables.  

In practice, when users feed data into Meridian, Meridian will automatically standardize the control variables before posterior sampling. Details about the data transformation is on the [documentation page](https://developers.google.com/meridian/docs/basics/input-data).

In [ ]:
# @title Standardize control variables

control_transformer = transformers.CenteringAndScalingTransformer(
    tensor=control_gtc, population=p_g, population_scaling_id=None
)
transformed_control_gtc = control_transformer.forward(control_gtc)

By default, Meridian does not population-scale control variables; we also do not do it here (as represented by the argument `population_scaling_id=None`). However, users have to determine if certain control variables may need to be population scaled; see this [documentation page](https://developers.google.com/meridian/docs/advanced-modeling/control-variables#population-scaling) for more details.  

## Media Channels

We then simulate the impression per capita $x^{\dagger}_{g,t,m}$ for each channel $m$ (these include both impression-based and rf-based channels).

$$x^{\dagger}_{g,t,m} = \max \left\{ u_{m} + u_{t,m} + u_{g,t,m} \,\, , \,\,0\right\} $$

* $u_m \overset{\mathrm{iid}}{\sim} N(1, 0.5)$,
* $u_{t,m} \overset{\mathrm{iid}}{\sim} N(0.8, 0.5)$
* $u_{g,t,m}\overset{\mathrm{iid}}{\sim} N(0, 0.5)$

The impression per capita per channel $x^{\dagger}_{g,t,m}$ is then multiplied with simulated population $p_g$ to obtain impression per channel $x_{g,t,m}$. We will simulate reach and frequency for RF channels later.

In [ ]:
# @title Simulate impression per channel

u_m = tfp.distributions.Normal(1, 0.5).sample(n_total_paid_channels)
u_tm = tfp.distributions.Normal(0.8, 0.5).sample(
    [n_times, n_total_paid_channels]
)
u_gtm = tfp.distributions.Normal(0, 0.5).sample(
    (n_geos, n_times, n_total_paid_channels)
)

# impression per capita per channel
ipc_gtm = np.maximum(u_m + u_tm + u_gtm, 0.0)

# impression per channel
impression_gtm = tf.round(ipc_gtm * p_gtm)

# Check percentage or sparsity
ipc_sparsity = np.sum(ipc_gtm == 0.0, axis=(0, 1)) / (n_geos * n_times)
ipc_sparsity_percentage = [f'{s * 100:.2f}%' for s in ipc_sparsity]
print(
    'percentage of sparsity of impression_per_capita for each channel:'
    f' {ipc_sparsity_percentage}'
)

We now simulate reach $r_{g,t,m}$ and frequency $f_{g,t,m}$ based on the previously simulated impression $x_{g,t,m}$. We adopt the following assumptions in our simulation:

*   Let $\theta$ be the expected reach per person. An estimate of $\theta$ is the impression-per-capita (ipc).
*   For each user, assume the reach frequency follows a Poisson distribution with parameter $\theta$. Then the probability that user is reached at least once is
$$
\mathbf{P}(\text{A person is reached at least once}) = 1 - \mathbf{P}(\text{never reached}) = 1 - e^{-\theta}.
$$
Replacing $\theta$ with ipc yields an estimate of this probability, which can be used in simulation.

* Assume individuals are independent from one another. Then the number of people reached has mean $n_{\rm pop}(1 - e^{-\theta})$ and variance $n_{\rm pop}e^{-\theta}(1 - e^{-\theta})$. To simulate total people reached, we draw from
$$
\text{total_reach} \sim N \left(n_{\rm pop}(1 - e^{-\theta}), n_{\rm pop}e^{-\theta}(1 - e^{-\theta}) \right).
$$
The reach-per-capita (rpc) would be
$$
\text{rpc} = \frac{\text{total_reach}}{n_{\rm pop}}.
$$

* The frequency would then be
$$
\text{frequency} = \frac{\text{impressions per capita (ipc)}}{\text{reach per capita (rpc)}}.
$$

In [ ]:
# @title Simulate reach and frequency

# impressions per capita for R&F channels in [geos, times, channels] dimensions.
ipc_gtm_rf = ipc_gtm[..., -n_rf_channels:]
impression_gtm_rf = impression_gtm[..., -n_rf_channels:]

# reach
reach_gtm_mean = p_gtm * (1 - tf.math.exp(-ipc_gtm_rf))
reach_gtm_scale = np.sqrt(
    p_gtm * (tf.math.exp(-ipc_gtm_rf) - tf.math.exp(-2 * ipc_gtm_rf))
)
reach_gtm = tf.round(
    tfp.distributions.Normal(reach_gtm_mean, reach_gtm_scale).sample()
)

# frequency
freq_gtm = np.where(impression_gtm_rf == 0, 0, impression_gtm_rf / reach_gtm)

Again, to ensure that the control variables and media variables are simulated at the same scale, we will transform the simulated media reach and impression data.    

In practice, when users feed data into Meridian, Meridian will automatically transform the media variables (including population scaling) before posterior sampling. Details about the data transformation is on the [documentation page](https://developers.google.com/meridian/docs/basics/input-data).

In [ ]:
# @title Transform media variables

# Transform impression
impression_gtm_imp_only = impression_gtm[..., :-n_rf_channels]
impression_transformer = transformers.MediaTransformer(
    media=impression_gtm_imp_only, population=p_g
)
transformed_ipc_gtm = impression_transformer.forward(impression_gtm_imp_only)

# Transform reach
reach_transformer = transformers.MediaTransformer(
    media=reach_gtm, population=p_g
)
transformed_rpc_gtm = reach_transformer.forward(reach_gtm)

## Time-varying intercepts ($\mu_t$) and geo effects ($\tau_g$)

$\tau_g \stackrel{iid}{\sim} N(15, 1.2)$

$\mu_t \stackrel{iid}{\sim} N(0, 2)$

Here, $\mu_t$ was simulated using [full knots](https://developers.google.com/meridian/docs/basics/model-spec#_mu_t_parameters), as specified by n_knots_simul = n_times. (For more information on how the knot arguments work, see this [documentation page](https://developers.google.com/meridian/docs/advanced-modeling/setting-knots#how-knots-argument-works).) If we want to simulate $\mu_t$ using fewer knots, we could set n_knots_simul to be a positive integer smaller than n_times. When we simulate $\mu_t$ with full knots, we could also simplify the codes to be 

```python
mu_t =  tfp.distributions.Normal(0, 2.0).sample(n_times).
```

In [ ]:
# @title Simulate mu_t and tau_g

tau_g = tfp.distributions.Normal(15.0, 1.2).sample(n_geos)

n_knots_simul = n_times
knots_k = tfp.distributions.Normal(0, 2.0).sample(n_knots_simul)
# Initialize Meridian's knots object (we need the weights)
knots_object = knots.get_knot_info(n_times, n_knots_simul, False)
# From simulated knots, get mu_t
mu_t = tfp.distributions.Deterministic(
    tf.einsum(
        '...k,kt->...t',
        knots_k,
        tf.convert_to_tensor(knots_object.weights),
    )
).sample()

plt.plot(mu_t)
plt.xlabel('time')
plt.ylabel('mu_t')
plt.show()

## Cost and unit value

$$
\text{cost_per_impression} \sim Unif (0.011, 0.012)
$$
$$
\text{unit_value} \sim Unif (0.0345, 0.0355)
$$

In [ ]:
# @title Simulate cost and unit value

CPM_m = tfp.distributions.Uniform(0.011, 0.012).sample(n_total_paid_channels)
cost_gtm = impression_gtm * CPM_m
unit_value = tfp.distributions.Uniform(0.0345, 0.0355).sample((n_geos, n_times))

# Plot the histograms
fig, axes = plt.subplots(n_total_paid_channels // 2, 2, figsize=(12, 6))
axes = axes.flatten()
for i, ax in enumerate(axes):
  ax.hist(tf.reshape(cost_gtm[..., i], [-1]), bins=30, alpha=0.7)
  ax.set_title(f'Histogram for Channel {i}')
  ax.set_xlabel('Values across all geos and times')
  ax.set_ylabel('Spend per channel')
plt.tight_layout()
plt.show()

## Coefficients ($\beta_{g,m}$ and $\gamma_{g,c}$) and error term $\epsilon_{g,t}$

For media coefficients,
* $\beta_m \stackrel{iid}{\sim} N(0.9, 0.1)$
* $\eta_m \stackrel{iid}{\sim} \text{HalfNormal}(0.18)$
* $\beta_{g,m} \big| (\beta_m, \eta_m) \stackrel{iid}{\sim} \text{LogNormal} (\beta_m, \eta_m)$.

For control coefficients,
* $\gamma_c \stackrel{iid}{\sim} N(3.5, 0.5)$
* $\xi_c \stackrel{iid}{\sim} \text{HalfNormal}(0.3)$
* $\gamma_{g,c} \big| (\gamma_c, \xi_c) \stackrel{iid}{\sim} N(\gamma_c, \xi_c)$.

For residual variance,
* $\epsilon_{g,t} \stackrel{iid}{\sim} N(0, 0.5)$.

In [ ]:
# @title Simulate coefficients and error term

# Media's coefficients
beta_m = tfp.distributions.Normal(0.9, 0.1).sample(n_total_paid_channels)
eta_m = tfp.distributions.HalfNormal(0.18).sample(n_total_paid_channels)
beta_gm_dev = tfp.distributions.Normal(0, 1).sample(
    [n_geos, n_total_paid_channels]
)
beta_gm = tf.exp(beta_m + eta_m * beta_gm_dev)

# Controls' coefficients
gamma_c = tfp.distributions.Normal(3.5, 0.5).sample(n_controls)
xi_c = tfp.distributions.HalfNormal(0.3).sample(n_controls)
gamma_gc_dev = tfp.distributions.Normal(0, 1).sample([n_geos, n_controls])
gamma_gc = gamma_c + xi_c * gamma_gc_dev

# epsilon
sigma = tf.fill([1], 0.5)
eps_gt = tfp.distributions.Normal(0, sigma[0]).sample([n_geos, n_times])

## HillAdstock Parameters

We simulate HillAdstock parameters (alpha's, ec's, slope's) from their corresponding Meridian's default prior distributions.

In [ ]:
#@title Simulate HillAdstock parameters

# Initialize prior distribution object
parameters=prior_distribution.PriorDistribution.broadcast(
    prior_distribution.PriorDistribution(),
    n_geos=n_geos,
    n_media_channels=n_imp_channels,
    n_controls=n_controls,
    n_rf_channels=n_rf_channels,
    sigma_shape=1,
    n_knots=1,
    is_national=False,
    n_organic_media_channels=0,
    n_organic_rf_channels=0,
    n_non_media_channels=0,
    set_total_media_contribution_prior=False,
    kpi=0,
    total_spend=0,
    media_effects_dist="normal"
)

# HillAdstock parameters for impression-based channels
alpha_m = parameters.alpha_m.sample()
ec_m = parameters.ec_m.sample()
slope_m = parameters.slope_m.sample()

# HillAdstock parameters for RF channels
alpha_rf = parameters.alpha_rf.sample()
ec_rf = parameters.ec_rf.sample()
slope_rf = parameters.slope_rf.sample()

Note that we want to make our simulation look plausible. We did not simulate all parameters from Meridian's default priors because our default priors have fairly "wide" distributions. One may end up drawing some strange combination of parameters from these wide distributions, which lead to a strange ROI.    

## KPI and revenue

In [ ]:
# @title Collecting HillAdstock terms for media channels

# HillAdstock term for impression-based channels
hill_transformer = model.adstock_hill.HillTransformer(ec=ec_m, slope=slope_m)
adstock_transformer = model.adstock_hill.AdstockTransformer(
    alpha=alpha_m,
    max_lag=8,
    n_times_output=n_times,
)
media_transformed = hill_transformer.forward(
    adstock_transformer.forward(transformed_ipc_gtm)
)

# HillAdstock term for RF channels
hill_transformer = model.adstock_hill.HillTransformer(ec=ec_rf, slope=slope_rf)
adstock_transformer = model.adstock_hill.AdstockTransformer(
    alpha=alpha_rf,
    max_lag=8,
    n_times_output=n_times,
)
adj_frequency = hill_transformer.forward(freq_gtm)
rf_transformed = adstock_transformer.forward(
    transformed_rpc_gtm * adj_frequency
)

# Both impression-based and RF-based media channels
media_rf_transformed = tf.concat([media_transformed, rf_transformed], axis=-1)

In [ ]:
# @title Generate KPI and Revenue

# KPI per capita
KPI_per_capita_gt = tf.maximum(
    tau_g[..., tf.newaxis]
    + tf.einsum("gtm,gm->gt", transformed_control_gtc, gamma_gc)
    + eps_gt
    + mu_t[tf.newaxis, ...],
    0.0,
) + tf.einsum("gtm,gm->gt", media_rf_transformed, beta_gm)

# KPI and revenue
KPI_gt = KPI_per_capita_gt * p_g[..., tf.newaxis]
Revenue_gt = KPI_gt * unit_value

# plot
plt.plot(np.mean(KPI_gt, axis=(0)))
plt.ylabel("KPI averaged over geos")
plt.xlabel("Time")
plt.show()

## Name arrays and convert into dataframe

In [ ]:
# @title Helping functions for naming arrays

def _sample_names(prefix: str, n_names: int | None = None) -> list[str]:
  """Generates a list of sample names.

  It concatenates the same prefix with consecutive numbers to generate a list
  of strings that can be used as sample names of columns/arrays/etc.
  """
  res = [prefix + str(n) for n in range(n_names)] if n_names else None
  return res


def _sample_times(
    n_times: int, start_date: datetime.date = datetime.date(2021, 1, 25)
) -> list[str]:
  """Generates sample `time`s."""
  res = [
      (start_date + datetime.timedelta(weeks=w)).strftime('%Y-%m-%d')
      for w in range(n_times)
  ]
  return res


def add_suffix(string: str, suffix: str) -> str:
  return '_'.join([string, suffix])


def remove_suffix(string: str, suffix: str) -> str:
  return string.replace('_' + suffix, '')


control_names = ['sentiment_score', 'competitor_activity_score']  # @param
if len(control_names) != n_controls:
  raise ValueError(f'Number of control names is not equal to {n_controls}')


CHANNEL_NAME_PREFIX = 'Channel'
GEO_NAME_PREFIX = 'Geo'

CHANNEL_DIM_NAME = 'channel'
RF_CHANNEL_DIM_NAME = 'rf_channel'
CONTROL_DIM_NAME = 'control'
GEO_DIM_NAME = GEO_COL_NAME = 'geo'
TIME_DIM_NAME = TIME_COL_NAME = 'time'

KPI_COL_NAME = 'conversions'
POPULATION_COL_NAME = 'population'
UNIT_VALUE_COL_NAME = 'revenue_per_conversion'

SPEND_COL_SUFFIX = 'spend'
IMPRESSIONS_COL_SUFFIX = 'impression'
REACH_COL_SUFFIX = 'reach'
FREQ_COL_SUFFIX = 'frequency'
CONTROL_COL_SUFFIX = 'control'

CONTROL_COL_NAMES = [add_suffix(c, CONTROL_COL_SUFFIX) for c in control_names]
CHANNEL_NAMES = _sample_names(CHANNEL_NAME_PREFIX, n_total_paid_channels)
GEO_NAMES = _sample_names(GEO_NAME_PREFIX, n_geos)
TIME_NAMES = _sample_times(n_times)

In [ ]:
# @title Combine tensors into an xarray or a Pandas DataFrame

media_data = xr.DataArray(
    impression_gtm,
    dims=[GEO_DIM_NAME, TIME_DIM_NAME, CHANNEL_DIM_NAME],
    coords={
        GEO_DIM_NAME: GEO_NAMES,
        TIME_DIM_NAME: TIME_NAMES,
        CHANNEL_DIM_NAME: CHANNEL_NAMES,
    },
    name=IMPRESSIONS_COL_SUFFIX,
)

control_data_name = 'control_value'
control_data = xr.DataArray(
    control_gtc,
    dims=[GEO_DIM_NAME, TIME_DIM_NAME, CONTROL_DIM_NAME],
    coords={
        GEO_DIM_NAME: GEO_NAMES,
        TIME_DIM_NAME: TIME_NAMES,
        CONTROL_DIM_NAME: CONTROL_COL_NAMES,
    },
    name=control_data_name,
)

spend_data = xr.DataArray(
    cost_gtm,
    dims=[GEO_DIM_NAME, TIME_DIM_NAME, CHANNEL_DIM_NAME],
    coords={
        GEO_DIM_NAME: GEO_NAMES,
        TIME_DIM_NAME: TIME_NAMES,
        CHANNEL_DIM_NAME: CHANNEL_NAMES,
    },
    name=SPEND_COL_SUFFIX,
)

kpi_data = xr.DataArray(
    KPI_gt,
    dims=[GEO_DIM_NAME, TIME_DIM_NAME],
    coords={GEO_DIM_NAME: GEO_NAMES, TIME_DIM_NAME: TIME_NAMES},
    name=KPI_COL_NAME,
)

unit_value_data = xr.DataArray(
    unit_value,
    dims=[GEO_DIM_NAME, TIME_DIM_NAME],
    coords={GEO_DIM_NAME: GEO_NAMES, TIME_DIM_NAME: TIME_NAMES},
    name=UNIT_VALUE_COL_NAME,
)

population_data = xr.DataArray(
    p_g,
    dims=[GEO_DIM_NAME],
    coords={GEO_DIM_NAME: GEO_NAMES},
    name=POPULATION_COL_NAME,
)

reach_data = xr.DataArray(
    reach_gtm,
    dims=[GEO_DIM_NAME, TIME_DIM_NAME, RF_CHANNEL_DIM_NAME],
    coords={
        GEO_DIM_NAME: GEO_NAMES,
        TIME_DIM_NAME: TIME_NAMES,
        RF_CHANNEL_DIM_NAME: CHANNEL_NAMES[-n_rf_channels:],
    },
    name=REACH_COL_SUFFIX,
)

frequency_data = xr.DataArray(
    freq_gtm,
    dims=[GEO_DIM_NAME, TIME_DIM_NAME, RF_CHANNEL_DIM_NAME],
    coords={
        GEO_DIM_NAME: GEO_NAMES,
        TIME_DIM_NAME: TIME_NAMES,
        RF_CHANNEL_DIM_NAME: CHANNEL_NAMES[-n_rf_channels:],
    },
    name=FREQ_COL_SUFFIX,
)

# DF
media_df = (
    media_data.to_dataframe()
    .reset_index()
    .pivot(
        index=[GEO_DIM_NAME, TIME_DIM_NAME],
        columns=CHANNEL_DIM_NAME,
        values=IMPRESSIONS_COL_SUFFIX,
    )
    .rename(columns=lambda x: add_suffix(x, IMPRESSIONS_COL_SUFFIX))
)

control_df = (
    control_data.to_dataframe()
    .reset_index()
    .pivot(
        index=[GEO_DIM_NAME, TIME_DIM_NAME],
        columns=CONTROL_DIM_NAME,
        values=control_data_name,
    )
)

spend_df = (
    spend_data.to_dataframe()
    .reset_index()
    .pivot(
        index=[GEO_DIM_NAME, TIME_DIM_NAME],
        columns=CHANNEL_DIM_NAME,
        values=SPEND_COL_SUFFIX,
    )
    .rename(columns=lambda x: add_suffix(x, SPEND_COL_SUFFIX))
)

kpi_df = kpi_data.to_dataframe().reset_index()

unit_value_df = unit_value_data.to_dataframe().reset_index()

population_df = population_data.to_dataframe().reset_index()

reach_df = (
    reach_data.to_dataframe()
    .reset_index()
    .pivot(
        index=[GEO_DIM_NAME, TIME_DIM_NAME],
        columns=RF_CHANNEL_DIM_NAME,
        values=REACH_COL_SUFFIX,
    )
    .rename(columns=lambda x: add_suffix(x, REACH_COL_SUFFIX))
)

frequency_df = (
    frequency_data.to_dataframe()
    .reset_index()
    .pivot(
        index=[GEO_DIM_NAME, TIME_DIM_NAME],
        columns=RF_CHANNEL_DIM_NAME,
        values=FREQ_COL_SUFFIX,
    )
    .rename(columns=lambda x: add_suffix(x, FREQ_COL_SUFFIX))
)

media_df.reset_index(inplace=True)
control_df.reset_index(inplace=True)
spend_df.reset_index(inplace=True)
reach_df.reset_index(inplace=True)
frequency_df.reset_index(inplace=True)

geo_data_df = (
    media_df.merge(control_df)
    .merge(spend_df)
    .merge(kpi_df)
    .merge(unit_value_df)
    .merge(population_df)
)

geo_rf_data_df = geo_data_df.merge(reach_df).merge(frequency_df)

In [ ]:
# @title Generate a national dataset from the geo dataframe

def _weighted_average(geo_df, freq_col_name):
  channel_name = remove_suffix(freq_col_name, FREQ_COL_SUFFIX)
  reach_col_name = add_suffix(channel_name, REACH_COL_SUFFIX)
  reach_col = geo_df[reach_col_name]
  total_reach = reach_col.sum()

  if total_reach == 0:
    # Handle cases where the sum of reach is zero to avoid division by zero.
    return 0

  return np.average(geo_df[freq_col_name], weights=reach_col)


def geo_df_to_national(geo_df):
  """Aggregates a pandas DataFrame by TIME_COL_NAME.

  Args:
      geo_df: The input DataFrame.

  Returns:
      A new DataFrame with aggregated data.
  """
  geo_df = geo_df.drop(columns=[POPULATION_COL_NAME])

  aggregations = {
      KPI_COL_NAME: 'sum',
      UNIT_VALUE_COL_NAME: 'mean',
  }

  # Add specific columns to the aggregation dictionary
  freq_cols = []

  for col in geo_df.columns:
    if col.endswith(IMPRESSIONS_COL_SUFFIX):
      aggregations[col] = 'sum'
    elif col.endswith(SPEND_COL_SUFFIX):
      aggregations[col] = 'sum'
    elif col.endswith(REACH_COL_SUFFIX):
      aggregations[col] = 'sum'
    elif col.endswith(CONTROL_COL_SUFFIX):
      aggregations[col] = 'mean'
    elif col.endswith(FREQ_COL_SUFFIX):
      freq_cols.append(col)

  # Aggregate the DataFrame
  national_df = geo_df.groupby(TIME_COL_NAME).agg(aggregations)

  for col in freq_cols:
    national_df[col] = geo_df.groupby(TIME_COL_NAME).apply(
        _weighted_average, freq_col_name=col, include_groups=False
    )

  return national_df


national_rf_data_df = geo_df_to_national(geo_rf_data_df).reset_index()

In [ ]:
# @title Collecting simulated parameters (on raw scale of KPI_per_capita)

dict_simul_param_on_raw_scale = {
    'alpha_m': alpha_m.numpy(),
    'alpha_rf': alpha_rf.numpy(),
    'beta_gm': beta_gm.numpy()[:, :n_imp_channels],
    'beta_grf': beta_gm.numpy()[:, -n_rf_channels:],
    'beta_m': beta_m.numpy()[:n_imp_channels],
    'beta_rf': beta_m.numpy()[-n_rf_channels:],
    'ec_m': ec_m.numpy(),
    'ec_rf': ec_rf.numpy(),
    'eta_m': eta_m.numpy()[:n_imp_channels],
    'eta_rf': eta_m.numpy()[-n_rf_channels:],
    'gamma_c': gamma_c.numpy(),
    'gamma_gc': gamma_gc.numpy(),
    'mu_t': mu_t.numpy(),
    'sigma': sigma.numpy(),
    'slope_m': slope_m.numpy(),
    'slope_rf': slope_rf.numpy(),
    'tau_g': tau_g.numpy(),
    'xi_c': xi_c.numpy(),
    'intercept_gt': (tau_g[..., tf.newaxis] + mu_t[tf.newaxis, ...]).numpy(),
}

## Ground-truth Parameters and ROI calculation

* First, recall that Meridian automatically transforms KPI data and media data (for more information, see [Input data](https://developers.google.com/meridian/docs/basics/input-data) documentation page).

* Thus, Meridian's posterior samples for model parameters should be interpreted on (the scales of) transformed KPI data and media data.

* Consequently, the simulated parameters used to generate raw KPI data does not have the same meaning as Meridian's model parameter fit on the transformed KPI data.

* We need to scale these simulated parameters accordingly, so that what we regard as the "ground-truth" parameters are on the same scale as Meridian's sampled parameters.

In [ ]:
# @title Obtain ground-truths to ensure comparison with posterior samples at the same scale

# Get the centering and scaling factors needed.
kpi_transformer = transformers.KpiTransformer(kpi=KPI_gt, population=p_g)
kpi_mean = kpi_transformer.population_scaled_mean.numpy().item()
kpi_stdev = kpi_transformer.population_scaled_stdev.numpy().item()

# Define the simulated parameters that need to be scaled.
param_to_scale = [
    'beta_gm',
    'beta_grf',
    'gamma_c',
    'gamma_gc',
    'sigma',
    'xi_c',
]

# Initialize
dict_ground_truth = {}

# Scale
for param in param_to_scale:
  dict_ground_truth[param] = dict_simul_param_on_raw_scale[param] / kpi_stdev

# Since we assume lognormal distribution for prior of beta_gm and beta_grf,
# we need to transform beta_m and beta_rf differently.
# Note: if log(X) ~ N(mu, sigma), then log(X/k) ~ N(mu-log(k), sigma).
for param in ['beta_m', 'beta_rf']:
  dict_ground_truth[param] = dict_simul_param_on_raw_scale[param] - np.log(
      kpi_stdev
  )

# We consider the geo-and-time intercept terms (mu_t + tau_g) together, due to
# the centering around kpi_mean.
dict_ground_truth['intercept_gt'] = (
    dict_simul_param_on_raw_scale['intercept_gt'] - kpi_mean
) / kpi_stdev

In [ ]:
# @title Compute incremental revenue and ROI

# ROI
# ROI formula is explained at https://developers.google.com/meridian/docs/basics/roi-and-mroi-parameterization#roi
Incremental_Revenue_m = kpi_stdev * tf.einsum(
    'g,gt,gtm,gm->m',
    p_g,
    unit_value,
    media_rf_transformed,
    tf.concat(
        [dict_ground_truth['beta_gm'], dict_ground_truth['beta_grf']], axis=1
    ),
)
ground_truth_roi = Incremental_Revenue_m / tf.einsum('gtm->m', cost_gtm)
total_incremental_roi = np.sum(Incremental_Revenue_m) / np.sum(cost_gtm)
total_revenue = np.sum(Revenue_gt)

print(f'ground-truth ROI for every channel = {ground_truth_roi.numpy()}')
print(f'ground-truth total incremental ROI = {total_incremental_roi:.2f}')
print()
print(f'Total revenue = {total_revenue / 1e6:.2f}M')
print(f'Total ROI = {total_revenue / np.sum(cost_gtm)}')

# Store in dictionary
dict_ground_truth['roi_m'] = ground_truth_roi.numpy()[:n_imp_channels]
dict_ground_truth['roi_rf'] = ground_truth_roi.numpy()[-n_rf_channels:]

# Save the simulated data and the ground-truth values

In [ ]:
geo_rf_data_df.to_csv(
    'geo_rf_data.csv', index=False
)
national_rf_data_df.to_csv(
    'national_rf_data.csv', index=False
)
with open('dict_ground_truth.pkl', 'wb') as f:
  pickle.dump(dict_ground_truth, f)